# <span style='color:green; font-family:Helvetica'> Classifying Pepper and Weeping Willow Trees </span>
### <span style='color:green; font-family:Helvetica'> by DeepSquad </span>

<img src="images_trees/ICON.png" alt="Image" style="width:500px;height:400px;">


## <span style='color:green; font-family:Helvetica'> Data Preparation

In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training dataset
train_set = ImageFolder('images_trees/train', transform=transform)

# Load the test dataset
test_set = ImageFolder('images_trees/test', transform=transform)


## <span style='color:green; font-family:Helvetica'> Model Selection

In [4]:
import torch
import torchvision.models as models

# Load a pre-trained model
model = models.resnet50(pretrained=True)

# Modify the last fully connected layer to match the number of classes
num_classes = 2
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)


/Users/chochu/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/chochu/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## <span style='color:green; font-family:Helvetica'> Model Fine-Tuning<span style='color:green; font-family:Helvetica'> 

In [3]:
# Set requires_grad to False for all parameters except the last layer
for param in model.parameters():
    param.requires_grad = False

# Set requires_grad to True for the parameters of the last layer
for param in model.fc.parameters():
    param.requires_grad = True


## <span style='color:green; font-family:Helvetica'> Training

In [6]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

# Create data loaders for training and testing
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")


## <span style='color:green; font-family:Helvetica'> Model Evaluation

In [5]:
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 95.74%


## <span style='color:green; font-family:Helvetica'> Saving the Model

In [6]:
torch.save(model.state_dict(), 'model.pt')
print("Model Saved")

Model Saved


## <span style='color:green; font-family:Helvetica'> Streamlit UI

In [ ]:
#Run on command line
#streamlit run tree_classification_server.py


## <span style='color:green; font-family:Helvetica'> Model Accuracy Without Fine Tuning

In [8]:
# Load a pre-trained model
model = models.resnet50(pretrained=True)

# Modify the last fully connected layer to match the number of classes
num_classes = 2
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 68.09%
